In [ ]:
#imports

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from keras.models import Sequential

from PIL import Image
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import shutil
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay,classification_report

from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
#connect google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#data directories

import pathlib
data_dir = "/content/drive/MyDrive/FlatironProjects/Phase-4/chest_xray"
data_dir_train = "/content/drive/MyDrive/FlatironProjects/Phase-4/chest_xray/train"
data_dir_test = "/content/drive/MyDrive/FlatironProjects/Phase-4/chest_xray/test"
data_dir_validation = "/content/drive/MyDrive/FlatironProjects/Phase-4/chest_xray/val"

In [ ]:
#check and pull images, checking for bad files

from pathlib import Path
import imghdr

data_dir = "/content/drive/MyDrive/FlatironProjects/Phase-4/chest_xray"
image_extensions = [".png", ".svg", ".gif", ".jpg"]  # add there all your images file extensions

img_type_accepted_by_tf = ["bmp", "gif", "jpeg", "png"]
for filepath in Path(data_dir).rglob("*"):
    if filepath.suffix.lower() in image_extensions:
        img_type = imghdr.what(filepath)
        if img_type is None:
            print(f"{filepath} is not an image")
        elif img_type not in img_type_accepted_by_tf:
            print(f"{filepath} is a {img_type}, not accepted by TensorFlow")

In [ ]:
#image augmentation
img_height, img_width = 64, 64
batch_size = 32

#process datasets

#train
train_ds = tf.keras.preprocessing.image_dataset_from_directory(data_dir_train,
                                                               label_mode = 'binary',
                                                               image_size=(img_height, img_width),
                                                               batch_size=batch_size)

#test
test_ds = tf.keras.preprocessing.image_dataset_from_directory(data_dir_test,
                                                              label_mode = 'binary',
                                                              image_size =(img_height, img_width),
                                                              batch_size=batch_size)

#validation
val_ds = tf.keras.preprocessing.image_dataset_from_directory(data_dir_validation,
                                                             label_mode = 'binary',
                                                             image_size=(img_height, img_width),
                                                             batch_size=batch_size)



#Creating three datases of images from a directory of image files. This function returns a tf.data.Dataset
#containing batches of images and their corresponding labels.
#Each object will contain image and label pairs that can be used to train a neural network.

Found 4729 files belonging to 2 classes.
Found 586 files belonging to 2 classes.
Found 16 files belonging to 2 classes.


In [ ]:
#class names
class_names = train_ds.class_names
print(class_names)

num_classes = 2

['NORMAL', 'PNEUMONIA']


In [ ]:
#check type
train_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

# Modeling

In [ ]:
#imports

from keras import models
from keras import layers
from keras import optimizers
from keras.layers import Activation

In [ ]:
#first model attempt, first simple model

fsm = models.Sequential()

#dense layer
fsm.add(layers.Dense(10, activation='tanh', input_shape=(64, 64, 3)))

#flatten
fsm.add(layers.Flatten())

#dense
fsm.add(layers.Dense(2, activation='sigmoid'))

In [ ]:
#compile
fsm.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
#This script essentially trains a neural network model on the training dataset,
# periodically logs training progress evaluates the model's performance on a separate test dataset after training.

#define epochs
epochs = 25

#For each epoch, it iterates over the batches of the training dataset,
## computes the model's predictions, calculates the loss, computes gradients,
## and updates the model's parameters using gradient descent.

iiterator = iter(train_ds)
max_iterations = len(train_ds)
for epoch in range(epochs):
    print("\nstart of epoch %d" % (epoch,))
    # Iterate over the batches of the dataset.
    i = 0
    while i < max_iterations:
        # print("currently running {} batch".format(i))  # Remove or comment out this line
        try:
            i = i + 1
            x_batch_train, y_batch_train = next(iterator)
            with tf.GradientTape() as tape:
                logits = model(x_batch_train, training=True)
                loss_value = loss_fn(y_batch_train, logits)

            grads = tape.gradient(loss_value, fsm.trainable_weights)
            optimizer.apply_gradients(zip(grads, fsm.trainable_weights))

            # log every 200 batches
            if i % 200 == 0:
                print("training loss at batch %d: %.4f" % (i, float(loss_value)) )
                print("seen so far: %s samples" % ((i + 1) * batch_size))

            train_acc = train_acc_metric.result()
            print("training accuracy per epoch: %.4f" % (float(train_acc),))

            # reset after each epoch
            train_acc_metric.reset_states()
            for x_batch_val, y_batch_val in val_ds:
                val_logits = model(x_batch_val, training=False)
                # Update val metrics
                val_acc_metric.update_state(y_batch_val, val_logits)
            val_acc = val_acc_metric.result()
            val_acc_metric.reset_states()
            print("Validation acc: %.4f" % (float(val_acc),))
        except Exception as e:
            continue

# eval
test_loss, test_accuracy = fsm.evaluate(test_ds)


start of epoch 0
currently running 0 batch
currently running 1 batch
currently running 2 batch
currently running 3 batch
currently running 4 batch
currently running 5 batch
currently running 6 batch
currently running 7 batch
currently running 8 batch
currently running 9 batch
currently running 10 batch
currently running 11 batch
currently running 12 batch
currently running 13 batch
currently running 14 batch
currently running 15 batch
currently running 16 batch
currently running 17 batch
currently running 18 batch
currently running 19 batch
currently running 20 batch
currently running 21 batch
currently running 22 batch
currently running 23 batch
currently running 24 batch
currently running 25 batch
currently running 26 batch
currently running 27 batch
currently running 28 batch
currently running 29 batch
currently running 30 batch
currently running 31 batch
currently running 32 batch
currently running 33 batch
currently running 34 batch
currently running 35 batch
currently running 36

/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5729: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


19/19 [==============================] - 130s 3s/step - loss: 0.7863 - accuracy: 0.4249


In [ ]:
input_shape = (img_height, img_width, 3)

In [ ]:
#second model

# Initialize a Sequential model
sm = Sequential()

In [ ]:
#convolutional layer 1 using relu
sm.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = (64, 64, 3)))
sm.add(MaxPooling2D(pool_size=(2, 2)))

#convolutional layer 2 using relu
sm.add(Conv2D(filters = 64, kernel_size=  (3, 3), activation = 'relu'))
sm.add(MaxPooling2D(pool_size = (2, 2)))

#flatten
sm.add(Flatten())

#dense layer 1
sm.add(Dense(units = 256, activation = 'relu'))


In [ ]:
#compile
sm.compile(
    loss='binary_crossentropy',  # Binary classification loss
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy'])
#summary of architecture
sm.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_21 (Conv2D)          (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_18 (MaxPooli  (None, 31, 31, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_22 (Conv2D)          (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_19 (MaxPooli  (None, 14, 14, 64)        0         
 ng2D)                                                           
                                                                 
 flatten_6 (Flatten)         (None, 12544)             0         
                                                                 
 dense_13 (Dense)            (None, 256)              

In [ ]:
pip install visualkeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.7/993.7 kB 8.8 MB/s eta 0:00:00


In [ ]:
from PIL import ImageFont  # For loading the font
import visualkeras       # For visualizing the Keras model

font = ImageFont.load_default()
visualkeras.layered_view(sm, to_file='model.png',
                         legend=True, font=font).show()

In [ ]:
# fit and run model

In [ ]:
#define epochs
epochs = 25

iterator = iter(train_ds)
max_iterations = len(train_ds)
for epoch in range(epochs):
    print("\nstart of epoch %d" % (epoch,))
    # Iterate over the batches of the dataset.
    i = 0
    while i < max_iterations:
        # print("currently running {} batch".format(i))  # Remove or comment out this line
        try:
            i = i + 1
            x_batch_train, y_batch_train = next(iterator)
            with tf.GradientTape() as tape:
                logits = model(x_batch_train, training=True)
                loss_value = loss_fn(y_batch_train, logits)

            grads = tape.gradient(loss_value, sm.trainable_weights)
            optimizer.apply_gradients(zip(grads, sm.trainable_weights))

            # log every 200 batches
            if i % 200 == 0:
                print("training loss at batch %d: %.4f" % (i, float(loss_value)) )
                print("seen so far: %s samples" % ((i + 1) * batch_size))

            train_acc = train_acc_metric.result()
            print("training accuracy per epoch: %.4f" % (float(train_acc),))

            # reset after each epoch
            train_acc_metric.reset_states()
            for x_batch_val, y_batch_val in val_ds:
                val_logits = model(x_batch_val, training=False)
                # Update val metrics
                val_acc_metric.update_state(y_batch_val, val_logits)
            val_acc = val_acc_metric.result()
            val_acc_metric.reset_states()
            print("Validation acc: %.4f" % (float(val_acc),))
        except Exception as e:
            continue




start of epoch 0

start of epoch 1

start of epoch 2

start of epoch 3

start of epoch 4

start of epoch 5

start of epoch 6

start of epoch 7

start of epoch 8

start of epoch 9

start of epoch 10

start of epoch 11

start of epoch 12

start of epoch 13

start of epoch 14

start of epoch 15

start of epoch 16

start of epoch 17

start of epoch 18

start of epoch 19

start of epoch 20

start of epoch 21

start of epoch 22

start of epoch 23

start of epoch 24


In [ ]:
# eval
test_loss, test_accuracy = sm.evaluate(test_ds)

19/19 [==============================] - 5s 90ms/step - loss: 7.5721 - accuracy: 0.0000e+00


In [ ]:
#third model

# Initialize a Sequential model
tm = Sequential()

In [ ]:
#convoltional layer 1 using sigmoid
tm.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = 'sigmoid', input_shape = (64, 64, 3)))
tm.add(MaxPooling2D(pool_size = (2, 2)))

# convoltional layer 2 using sigmoid increased filters
tm.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'sigmoid'))
tm.add(MaxPooling2D(pool_size=(2, 2)))

#convoltional layer 3 using sigmoid increased filters
tm.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = 'sigmoid'))
tm.add(MaxPooling2D(pool_size = (2, 2)))

#convoltional layer 4 using sigmoid increased filters
tm.add(Conv2D(filters = 256, kernel_size = (3, 3), activation = 'sigmoid'))
tm.add(MaxPooling2D(pool_size = (2, 2)))

#flatten
tm.add(Flatten())

#dense 1
tm.add(Dense(units = 256, activation = 'sigmoid'))

#dropout for regularization
tm.add(Dropout(0.5))

#dense 2
tm.add(Dense(units=128, activation = 'sigmoid'))

#dropout for regularization
tm.add(Dropout(0.3))

#dense 3
tm.add(Dense(units=10, activation = 'softmax'))


In [ ]:
#compile
tm.compile(loss='sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

#summary of architecture
tm.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_14 (MaxPooli  (None, 31, 31, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_16 (Conv2D)          (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_15 (MaxPooli  (None, 14, 14, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_17 (Conv2D)          (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_16 (MaxPooli  (None, 6, 6, 128)        

In [ ]:
#define epochs
epochs = 25

iterator = iter(train_ds)
max_iterations = len(train_ds)
for epoch in range(epochs):
    print("\nstart of epoch %d" % (epoch,))
    # Iterate over the batches of the dataset.
    i = 0
    while i < max_iterations:
        # print("currently running {} batch".format(i))  # Remove or comment out this line
        try:
            i = i + 1
            x_batch_train, y_batch_train = next(iterator)
            with tf.GradientTape() as tape:
                logits = model(x_batch_train, training=True)
                loss_value = loss_fn(y_batch_train, logits)

            grads = tape.gradient(loss_value, tm.trainable_weights)
            optimizer.apply_gradients(zip(grads, tm.trainable_weights))

            # log every 200 batches
            if i % 200 == 0:
                print("training loss at batch %d: %.4f" % (i, float(loss_value)) )
                print("seen so far: %s samples" % ((i + 1) * batch_size))

            train_acc = train_acc_metric.result()
            print("training accuracy per epoch: %.4f" % (float(train_acc),))

            # reset after each epoch
            train_acc_metric.reset_states()
            for x_batch_val, y_batch_val in val_ds:
                val_logits = model(x_batch_val, training=False)
                # Update val metrics
                val_acc_metric.update_state(y_batch_val, val_logits)
            val_acc = val_acc_metric.result()
            val_acc_metric.reset_states()
            print("Validation acc: %.4f" % (float(val_acc),))
        except Exception as e:
            continue


start of epoch 0

start of epoch 1

start of epoch 2

start of epoch 3

start of epoch 4

start of epoch 5

start of epoch 6

start of epoch 7

start of epoch 8

start of epoch 9

start of epoch 10

start of epoch 11

start of epoch 12

start of epoch 13

start of epoch 14

start of epoch 15

start of epoch 16

start of epoch 17

start of epoch 18

start of epoch 19

start of epoch 20

start of epoch 21

start of epoch 22

start of epoch 23

start of epoch 24


In [ ]:
# eval
test_loss, test_accuracy = tm.evaluate(test_ds)

19/19 [==============================] - 4s 85ms/step - loss: 2.7878 - accuracy: 0.0000e+00
